In [1]:
import numpy as np
import json
from flask import Flask, request, jsonify, render_template
import pandas as pd
import numpy as np
import pvlib

In [5]:
def clear_sky():
    latitude = -33.8878872
    longitude = 151.2007315
    start_date = '2020-01-01'
    end_date = '2020-01-02'
    capacity = 100

    location = pvlib.location.Location(latitude, longitude, tz='Australia/Sydney')

    times = pd.date_range(start=start_date, end=end_date, freq='1H', tz='Australia/Sydney')
    solpos = location.get_solarposition(times=times)
    dni_extra = pvlib.irradiance.get_extra_radiation(times)
    airmass = pvlib.atmosphere.get_relative_airmass(solpos['apparent_zenith'])
    pressure = pvlib.atmosphere.alt2pres(location.altitude)
    am_abs = pvlib.atmosphere.get_absolute_airmass(airmass, pressure)
    tl = pvlib.clearsky.lookup_linke_turbidity(times, latitude, longitude)

    solis_clearsky = pvlib.clearsky.simplified_solis(solpos['apparent_zenith'], am_abs, tl)
    cs = location.get_clearsky(times, model='simplified_solis')

    capacity = float(capacity)

    system = pvlib.pvsystem.PVSystem(surface_tilt=30, surface_azimuth=180,
                                     module_parameters={'pdc0': capacity, 'gamma_pdc': -0.004},
                                     inverter_parameters={'pdc0': capacity},
                                     modules_per_string=1, strings_per_inverter=1,
                                     temperature_model_parameters={'a': -3.56, 'b': -0.075, 'deltaT': 3})
    mc = pvlib.modelchain.ModelChain(system, location, spectral_model='no_loss', aoi_model='no_loss')

    mc.run_model(solis_clearsky)

    # Replace NaN values in the 'ac' column with 0
    mc.ac.fillna(0, inplace=True)

    return mc
    

In [7]:
aa = clear_sky()

c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\pvlib\modelchain.py:511: pvlibDeprecationWarning: ModelChain.ac is deprecated and will be removed in v0.10. Use ModelChain.results.ac instead
  warnings.warn(msg, pvlibDeprecationWarning)


In [17]:
aa.ac.values.tolist()

c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\pvlib\modelchain.py:511: pvlibDeprecationWarning: ModelChain.ac is deprecated and will be removed in v0.10. Use ModelChain.results.ac instead
  warnings.warn(msg, pvlibDeprecationWarning)


[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.29934939546724537,
 6.3537777679600405,
 12.767802514482204,
 16.248709525390943,
 16.257803363775185,
 12.73519031256454,
 9.293585184661623,
 13.008782932890531,
 16.34637496673007,
 16.14226911424045,
 12.46211059124388,
 5.939298172159064,
 0.10783020005853403,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [19]:
aa.ac.to_dict()

{Timestamp('2020-01-01 00:00:00+1100', tz='Australia/Sydney', freq='H'): 0.0,
 Timestamp('2020-01-01 01:00:00+1100', tz='Australia/Sydney', freq='H'): 0.0,
 Timestamp('2020-01-01 02:00:00+1100', tz='Australia/Sydney', freq='H'): 0.0,
 Timestamp('2020-01-01 03:00:00+1100', tz='Australia/Sydney', freq='H'): 0.0,
 Timestamp('2020-01-01 04:00:00+1100', tz='Australia/Sydney', freq='H'): 0.0,
 Timestamp('2020-01-01 05:00:00+1100', tz='Australia/Sydney', freq='H'): 0.0,
 Timestamp('2020-01-01 06:00:00+1100', tz='Australia/Sydney', freq='H'): 0.0,
 Timestamp('2020-01-01 07:00:00+1100', tz='Australia/Sydney', freq='H'): 0.29934939546724537,
 Timestamp('2020-01-01 08:00:00+1100', tz='Australia/Sydney', freq='H'): 6.3537777679600405,
 Timestamp('2020-01-01 09:00:00+1100', tz='Australia/Sydney', freq='H'): 12.767802514482204,
 Timestamp('2020-01-01 10:00:00+1100', tz='Australia/Sydney', freq='H'): 16.248709525390943,
 Timestamp('2020-01-01 11:00:00+1100', tz='Australia/Sydney', freq='H'): 16.25780

In [21]:
jsonify({'result':aa.ac.to_dict()})

SyntaxError: invalid syntax (2893559713.py, line 1)